# Create CSV file for Review data

In [1]:
from amazon_scraper import AmazonScraper
scraper = AmazonScraper()
asin = "B09C6HGX1W"  # Example ASIN

product = scraper.scrape_product_reviews(asin)
if product:
    print(f"Product: {product.name}")
    print(f"Overall Rating: {product.overall_rating}")
    print(f"Total Reviews: {product.total_review_count}")
    print(f"\nFirst {len(product.review_list)} reviews:")
    for review in product.review_list:
        print(f"\nReview by {review.country} on {review.date}")
        print(f"Rating: {review.rating}")
        print(f"Title: {review.title}")
        print(f"Verified Purchase: {review.verified_purchase}")
        print(f"Found Helpful: {review.found_helpful}")

Scraping reviews sorted by: recent
Scraping page 1 for sort by recent and star rating AmazonFilterStarRating.ALL_STAR and format type AmazonFilterFormatType.ALL_FORMATS
Page 1 scraped successfully
HTML length: 442682
Found 10 review elements
Scraping page 2 for sort by recent and star rating AmazonFilterStarRating.ALL_STAR and format type AmazonFilterFormatType.ALL_FORMATS
Page 2 scraped successfully
HTML length: 435486
Found 10 review elements
Scraping page 3 for sort by recent and star rating AmazonFilterStarRating.ALL_STAR and format type AmazonFilterFormatType.ALL_FORMATS
Page 3 scraped successfully
HTML length: 450327
Found 10 review elements
Scraping page 4 for sort by recent and star rating AmazonFilterStarRating.ALL_STAR and format type AmazonFilterFormatType.ALL_FORMATS
Page 4 scraped successfully
HTML length: 444712
Found 10 review elements
Scraping page 5 for sort by recent and star rating AmazonFilterStarRating.ALL_STAR and format type AmazonFilterFormatType.ALL_FORMATS
Pag